In [35]:
import numpy as np
def v(x, y, z=0):
    return np.array([x, y, z])

import tempfile
from pathlib import Path

from solid import *
from solid.utils import *
import viewscad

from subprocess import run


R=viewscad.Renderer()

wall_size=2
fanZ=15
fanX=58
fanY=55
fan=v(fanX, fanY, fanZ)

bedZ=17
bedX=50
bedY=100
bed=v(bedX, bedY, bedZ)

def XYZ(X=0, Y=0, Z=0):
    return lambda *args : translate(v(X, Y, Z))(*args)

def NULLOBJ():
    return cube(0.1)

def render_to_stl(name, object):
    tdir=tempfile.TemporaryDirectory()
    dirpath=Path(tdir.name)
    stlname = Path(name+".stl")
    scadname = dirpath / (name + ".scad")
    print(f'{stlname}')
    print(f'{scadname}')
    # generate valid openscad code and store it in file
    scad_render_to_file(object, scadname.as_posix())

    # run openscad and export to stl
    print(f'openscad --render -o {stlname.name} {scadname.name}')
    run(["openscad", "--render", "-o",  stlname.name, scadname.as_posix()])
    tdir.cleanup()

def make_part(name, object, r=R):
    render_to_stl(name, object)
    return object



In [39]:
@bom_part("Clamp Jaw")
def clamp_jaw():
    part = NULLOBJ()
    return make_part("clamp_jaw", part)

@bom_part("Clamp Screw")
def clamp_screw():
    part = NULLOBJ()
    return make_part("clamp_screw", part)      

@bom_part("Fan Control Box")
def fanCTL(rotation=v(0, 0, 0), offset=v(0, 0, 0)):
    part = translate(offset) (
               rotate(rotation) (
                   cube(fan + v(wall_size, 0, wall_size), center=False)
               )
            )
    return make_part("fanctl", part)

def bedCTL(rotation=v(0, 0, 0), offset=v(0, 0, 0)):
    outer = cube(bed)
    inner = XYZ(wall_size, 2*wall_size, wall_size)(
                cube(bed - v(2*wall_size, 0, 2*wall_size)))
    part = translate(offset) (
               rotate(rotation)(
                   outer - inner
               )
            )
    return make_part("bedctl", part)

@bom_part("Base and Hook")
def base_and_hook():
    part = NULLOBJ()
    return make_part("base_and_hook", part)

def make_all_parts():
    base_and_hook()
    fanCTL()
    bedCTL()
    clamp_screw()
    clamp_jaw()


In [41]:
make_all_parts()

base_and_hook.stl
/tmp/tmpz3gdbx72/base_and_hook.scad
openscad --render -o base_and_hook.stl base_and_hook.scad
fanctl.stl
/tmp/tmp7iuhgb8k/fanctl.scad
openscad --render -o fanctl.stl fanctl.scad


Geometries in cache: 1
Geometry cache size in bytes: 728
CGAL Polyhedrons in cache: 0
CGAL cache size in bytes: 0
Total rendering time: 0:00:00.000
   Top level object is a 3D object:
   Facets:          6
Geometries in cache: 3
Geometry cache size in bytes: 2184
CGAL Polyhedrons in cache: 0
CGAL cache size in bytes: 0
Total rendering time: 0:00:00.000
   Top level object is a 3D object:
   Facets:          6
Geometries in cache: 3
Geometry cache size in bytes: 2184
CGAL Polyhedrons in cache: 3
CGAL cache size in bytes: 67056
Total rendering time: 0:00:00.021
   Top level object is a 3D object:
   Simple:        yes
   Vertices:       16
   Halfedges:      48
   Edges:          24
   Halffacets:     22
   Facets:         11
   Volumes:         2


bedctl.stl
/tmp/tmp6y2jubhl/bedctl.scad
openscad --render -o bedctl.stl bedctl.scad
clamp_screw.stl
/tmp/tmpae1u8y_k/clamp_screw.scad
openscad --render -o clamp_screw.stl clamp_screw.scad
clamp_jaw.stl
/tmp/tmp5cip23cb/clamp_jaw.scad
openscad --render -o clamp_jaw.stl clamp_jaw.scad


Geometries in cache: 1
Geometry cache size in bytes: 728
CGAL Polyhedrons in cache: 0
CGAL cache size in bytes: 0
Total rendering time: 0:00:00.000
   Top level object is a 3D object:
   Facets:          6
Geometries in cache: 1
Geometry cache size in bytes: 728
CGAL Polyhedrons in cache: 0
CGAL cache size in bytes: 0
Total rendering time: 0:00:00.000
   Top level object is a 3D object:
   Facets:          6
